# TO GET STARTED: Enter a mission id below and then select Cell->Run All from the menu above

In [1]:
mission_id = 25614

In [2]:
import numpy as np
import os
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
from urllib.parse import urlparse
from urllib import parse

%matplotlib inline
FOLLOWER_URL = os.environ['FOLLOWER_URL']
                          
def make_connection():
    url_output = FOLLOWER_URL
    url = urlparse(url_output)
    conn = psycopg2.connect(
        database=url.path[1:],
        user=url.username,
        password=url.password,
        host=url.hostname
    )
    return conn

In [3]:
def get_response(answer, transcription, q_type):
    if q_type in ['TextQuestion', 'TitleQuestion']:
        return str(answer[0])
    else:
        if transcription is not None:
            return str(transcription)
        else:
            return ""
cnc = make_connection()

snippets = pd.read_sql_query(
    """select id as snippet_id from snippets where mission_id = """
    + str(mission_id),
    cnc
)
snippet_list = (
    "(" + ",".join([str(a) for a in snippets["snippet_id"].tolist()]) + ")"
)
questions = pd.read_sql_query(
    """select id as question_id, position + 1 as question_num, type, part_id from questions where mission_id = """
    + str(mission_id) + """ and type in ('TitleQuestion','TextQuestion','VideoQuestion')"""
    ,
    cnc
)
question_list = (
    "(" + ",".join([str(a) for a in questions["question_id"].tolist()]) + ")"
)

part_list = (
    "(" + ",".join([str(a) for a in questions["part_id"].tolist()]) + ")"
)

parts = pd.read_sql_query(
    """select id as part_id, position + 1 as part_num from parts
    where id in """+ part_list,
    cnc
)
questions = pd.merge(questions, parts, on='part_id')
responses = pd.read_sql_query(
    """select id as response_id, question_id, answers from responses where snippet_id in """
    + snippet_list + """ and question_id in """ + question_list,
    cnc,
)

responses = pd.merge(responses, questions, on='question_id')
video_response_list = (
    "(" + ",".join([
        str(a) for a in responses[responses['type'] == 'VideoQuestion']
        ["response_id"].tolist()]) 
    + ")"
)  

transcripts = pd.read_sql_query(
    """select transcription, response_id from videos where
    response_id in """+ video_response_list,
    cnc
)
responses = pd.merge(
    responses,
    transcripts,
    on='response_id',
    how ='left'
)
responses['response'] = responses.apply(
    lambda row: get_response(row['answers'],row['transcription'], row['type']),
    axis=1
)
responses['word_count'] = responses.apply(
    lambda row: len(row['response'].split()),
    axis=1
)
cnc.close

<function connection.close>

# WORD COUNT STATS BY QUESTION

In [4]:
responses.groupby(['part_num','question_num'])['word_count'].agg(['mean','median','std','sum','count']).reset_index()

,part_num,question_num,mean,median,std,sum,count
0,2,2,3.498436,3,2.089278,3355,959
1,2,5,138.695925,143,41.720460,132732,957
2,3,1,2.986152,3,1.240882,14232,4766
3,3,9,28.183069,22,24.589583,26633,945
4,4,1,2.985027,3,1.144669,2791,935
5,4,3,147.529412,147,43.745700,137940,935


# WOUNT COUNT STATS BY PART

In [5]:
responses.groupby(['part_num'])['word_count'].agg(['mean','median','std','sum','count']).reset_index()

,part_num,mean,median,std,sum,count
0,2,71.026618,11,73.777301,136087,1916
1,3,7.155489,3,13.745331,40865,5711
2,4,75.257219,6,78.632375,140731,1870


# WORD COUNT STATS BY QUESTION TYPE

In [6]:
responses.groupby(['type'])['word_count'].agg(['mean','median','std','sum','count']).reset_index()

,type,mean,median,std,sum,count
0,TextQuestion,28.183069,22,24.589583,26633,945
1,TitleQuestion,3.059760,3,1.394980,20378,6660
2,VideoQuestion,143.061311,145,42.949781,270672,1892


# WORD COUNT STATS BY MISSION

In [7]:
responses['word_count'].agg(['mean','median','std','sum','count']).reset_index()

,index,word_count
0,mean,33.450879
1,median,3.000000
2,std,58.933327
3,sum,317683.000000
4,count,9497.000000
